In [1]:
import os
import re
import librosa
import soundfile as sf
import numpy as np

In [8]:
# --- 설정: 경로 지정 ---
wav_paths = [
    "unwelcomeSchool/vocals.wav",
    "unwelcomeSchool/bass.wav",
    "unwelcomeSchool/drums.wav",
    "unwelcomeSchool/other.wav"
]
osu_input = "UnwelcomeSchool_Hard.txt"
osu_output = "UnwelcomeSchool_Hard_modified.osu"
output_dir = os.path.join(os.path.dirname(osu_output), "splited_notes")
os.makedirs(output_dir, exist_ok=True)

In [9]:
# 증폭할 트랙 인덱스와 증폭 비율 (예: drums(2), other(3)만 증폭)
amplify_indices = [3]
amplify_factor = 1.5

# 페이드 인/아웃 길이 (ms)
fade_ms = 10
# --- 1) WAV 파일 로드 및 증폭 적용 ---
waves = []
sr = None
for idx, p in enumerate(wav_paths):
    y, sr_tmp = librosa.load(p, sr=None)
    if sr is None:
        sr = sr_tmp
    elif sr != sr_tmp:
        raise ValueError("WAV 파일 간 샘플링 레이트 불일치")
    # 전체 파일에 대해 증폭
    if idx in amplify_indices:
        y = np.clip(y * amplify_factor, -1.0, 1.0)
    waves.append(y)

In [ ]:
# --- 2) osu 파일 파싱 ---
with open(osu_input, 'r', encoding='utf-8') as f:
    osu_text = f.read()

In [ ]:
# Events 및 HitObjects 파싱
events_match = re.search(r"\[Events\](.*?)(?=\n\[|$)", osu_text, re.S)
hit_match = re.search(r"\[HitObjects\](.*)$", osu_text, re.S)
hit_lines = []
if hit_match:
    hit_lines = [l for l in hit_match.group(1).strip().splitlines() if l.strip()]
else:
    raise ValueError("[HitObjects] 섹션을 찾을 수 없습니다.")

In [ ]:
# --- 3) 페이드 윈도우 생성 ---
fade_len = int(sr * fade_ms / 1000)
ramp = np.linspace(0, 1, fade_len)

In [ ]:
# --- 4) 클립 추출 및 노트 적용 ---
clip_ms = 100
clip_len = int(sr * clip_ms / 1000)
# silent_tracks: 원본 복사본 -> 클립 구간만 무음 및 페이드 처리
silent_tracks = [w.copy() for w in waves]

new_events = []
new_hit_lines = []
cnt = 1

In [ ]:
for line in hit_lines:
    parts = line.split(',')
    time_ms = int(parts[2])
    tail = parts[-1]
    effects, filename = (tail.rsplit(':', 1) if ':' in tail else (tail, ''))

    start = int(sr * time_ms / 1000)
    end = min(start + clip_len, len(waves[0]))

    # 에너지 비교
    energies = [np.sum(w[start:end]**2) for w in waves]
    idx = int(np.argmax(energies))

    # 클립 추출
    clip = waves[idx][start:end].copy()
    # 페이드 인/아웃 적용
    if len(clip) > 2 * fade_len:
        clip[:fade_len] *= ramp
        clip[-fade_len:] *= ramp[::-1]
    else:
        # 짧을 경우 전체에 램프 적용
        clip *= np.linspace(0, 1, len(clip))

    # silent_tracks에서 페이드 아웃 처리 후 0 대체
    seg = silent_tracks[idx][start:end].copy()
    if len(seg) > 2 * fade_len:
        seg[:fade_len] *= 1 - ramp
        seg[-fade_len:] *= 1 - ramp[::-1]
    silent_tracks[idx][start:end] = 0

    # 클립 저장
    clip_name = f"note_{idx+1}_{cnt}.wav"
    sf.write(os.path.join(output_dir, clip_name), clip, sr)

    # HitObjects 업데이트
    parts[-1] = f"{effects}:{clip_name}"
    new_hit_lines.append(','.join(parts))

    # Events 업데이트
    new_events.append(f"Sample,{time_ms},0,\"{clip_name}\",50")
    cnt += 1

In [ ]:
# --- 5) accompaniment 생성 ---
accompaniment = np.zeros_like(waves[0])
for tr in silent_tracks:
    accompaniment += tr
sf.write(os.path.join(output_dir, "accompaniment.wav"), accompaniment, sr)

In [ ]:
# --- 6) 수정된 osu 파일 작성 ---
header = osu_text[:events_match.start()]
between = osu_text[events_match.end():hit_match.start()]
footer = osu_text[hit_match.end():]

with open(osu_output, 'w', encoding='utf-8') as f:
    f.write(header)
    f.write("[Events]\n")
    for ev in new_events:
        f.write(ev + "\n")
    f.write("\n" + between)
    f.write("[HitObjects]\n")
    for hl in new_hit_lines:
        f.write(hl + "\n")
    f.write(footer)

print(f"완료: 추출, 증폭, 페이드 처리 및 accompaniment 생성 완료. 파일 위치: {output_dir}")
